## PHYS64 Spring 2024
### Lev Gruber

## Ammonia Cloud Computing Notebook
Code was built from scratch using Qiskit documentation while working off ideas in this paper: https://arxiv.org/pdf/2312.04230.pdf

In [ ]:
# Import all necessary packages

# General packages / building molecule
import qiskit
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
import csv

# CAS packages
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitMapper

# VQE usage packages
import qiskit_ibm_runtime
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD, UCC
from qiskit.circuit.library import EfficientSU2
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, SLSQP #optimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms.utils import algorithm_globals
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# On PC simulation packages
from qiskit_aer import Aer

# Cloud simultation packages
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV1 as Estimator
from qiskit_ibm_provider import IBMProvider

# Cloud Simulation Blocks

#### 1. Finds the GSE of a single nitrogen distance using a chosen ibm simulator, created first to determine necessary dependencies

In [ ]:
'''
Block to run a single calculation of ground state energy at a given N distance

To adjust N distance, change the third value after 'N' in atom = within the PySCF driver call.

To use any available simulator, comment first backend line and uncomment second

Known bug: Some simulators do not agree with uccsd architecture so transpiler fails and you get:
TranspilerError: "The number of qubits for Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]) does not match the number of qubits in the properties dictionary: (0,)"
Solution is to simply rerun until you happen upon a simulator that works with it.
'''

service = QiskitRuntimeService(channel="ibm_quantum") #, token = 'INSERT API TOKEN'
#backend = service.get_backend("ibm_kyoto")
backend = service.least_busy(operational=True, simulator=True)
print(backend.name)
with Session(service=service, backend=backend) as session:
    
    estimator = Estimator(backend=backend)
    estimator.options.default_shots = 1000
    
    # Use PySCF to compute 'one-body and two-body integrals in electronic orbital basis'.. or more simply build the particle
    driver = PySCFDriver(
        atom = 'N 0.0 0.0 0.0; H -0.8121 -0.4689 0.0; H 0.8121 -0.4689 0.0; H 0.0 0.9377 0.0' , 
        unit=DistanceUnit.ANGSTROM,         # define the above distances to be in terms of Angstroms (meaning for ex. .5 on N is 5*10^-11 m)
        basis='sto6g' 
    )

    problem = driver.run()

    # define 2e 2o transformer and use it to redefine problem w/ CAS(2o, 2e)
    transformer = ActiveSpaceTransformer(2, 2)
    as_problem = transformer.transform(problem)

    mapper = JordanWignerMapper()

    # Convert the problem to a qubit operator
    qubit_op = mapper.map(as_problem.second_q_ops()[0])
    num_qubits = qubit_op.num_qubits

    # setup VQE using Hartree-Fock as ansatz
    ansatz = UCCSD(
        as_problem.num_spatial_orbitals,
        as_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            as_problem.num_spatial_orbitals,
            as_problem.num_particles,
            mapper
        )
    )
    
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    isa_circuit = pm.run(ansatz)
    isa_observable = qubit_op.apply_layout(isa_circuit.layout)

    # Set up the optimizer
    optimizer = COBYLA(maxiter=1000)

    # Run VQE with custom ansatz
    vqe = VQE(estimator, isa_circuit, optimizer)
    vqe.backend = backend
    vqe.formatting_precision = 6
    vqe_result = vqe.compute_minimum_eigenvalue(isa_observable)
    
    # Run exact eigensolver locally for comparison
    numpy_solver = NumPyMinimumEigensolver()
    exact_calc = GroundStateEigensolver(mapper, numpy_solver)
    exact_energy = exact_calc.solve(as_problem)
    
    # Adjust vqe result to add repulsion energy and active space transformation energy
    vqe_energy = result.eigenvalue + exact_energy.total_energies - exact_energy.computed_energies
    
    session.close()


In [ ]:
# Save data in a csv
# Transpose lists to match csv format

distances = ['0.0']
rows = zip(distances, vqe_energy, exact_energy.total_energies)
with open('INSERTBACKEND_INSERTDISTANCES_INSERTDATE.csv', 'w') as f:
    writer = csv.writer(f)
    # Write the headers
    writer.writerow(['Distances', 'VQE Energies', 'Exact Energies'])
    # Write the data
    for row in rows:
        writer.writerow(row)

#### 2. Functions to call for determination of GSE in separation notebook block.
- get_inputs takes in the molecule and backend and returns ammonia-specific circuit and problem
- run_VQE runs the vqe on given backend and returns results
- find_energy is a wrapper for run vqe adn get_inputs, allowing the user to insert only nitrogen distances and a backend

In [ ]:
'''
Function block: Reorganzing above block into callable functions to standardize how both simulation and quantum computer code will run.

Written by Lev Gruber from scratch and qiskit documentation.
'''
def get_inputs(molecule, backend):
    '''
    Using an input molecule, returns the circuit and problem used for the vqe
    
    Parameters:
    - molecule: molecule following PySCF design.
    - backend: IBM quantum backend.
    - optional:
    - estimator and mapper.
    
    Returns:
    isa_circuit: ammonia hamiltonian ansatz mapped on quantum circuit for vqe use.
    problem: problem the ansatz uses
    '''
    mapper = JordanWignerMapper()
     
    # Use PySCF to compute 'one-body and two-body integrals in electronic orbital basis'.. or more simply build the particle
    driver = PySCFDriver(
        atom = molecule, 
        unit=DistanceUnit.ANGSTROM,         # define the above distances to be in terms of Angstroms (meaning for ex. .5 on N is 5*10^-11 m)
        basis='sto6g' 
    )

    problem = driver.run()

    # define 2e 2o transformer and use it to redefine problem w/ CAS(2o, 2e) ~ allows calculated energy to go from ~55 mH -> ~1.4 mH
    transformer = ActiveSpaceTransformer(2, 2)
    as_problem = transformer.transform(problem)
    
    # Convert the problem to a qubit operator
    qubit_op = mapper.map(as_problem.second_q_ops()[0])
    num_qubits = qubit_op.num_qubits

    # find the ansatz using Hartree-Fock method
    ansatz = UCCSD(
        as_problem.num_spatial_orbitals,
        as_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            as_problem.num_spatial_orbitals,
            as_problem.num_particles,
            mapper,
        )
    )
    
    # transile ansatz into circuit usable by cloud computer
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    isa_circuit = pm.run(ansatz)
    isa_observable = qubit_op.apply_layout(isa_circuit.layout)
    
    return isa_circuit, isa_observable, as_problem
    
def run_VQE(isa_circuit, isa_observable, as_problem, backend):
    """
    run the vqe algorithm.

    Parameters:
    - isa_circuit: The initial state ansatz circuit.
    - isa_observable: Operator of observable (energy)
    - as_problem: The vqe problem to solve.

    Returns:
    - vqe_result: The result of the VQE algorithm.
    - exact_result: The result of the exact algorithm.
    """
    estimator=Estimator(backend=backend)
    optimizer=COBYLA(maxiter=1000)
    mapper = JordanWignerMapper()
    
    # instantiate vqe and properties
    vqe = VQE(estimator, isa_circuit, optimizer)
    vqe.backend = backend
    vqe.formatting_precision = 6
    
    # solve exact result for comparison
    numpy_solver = NumPyMinimumEigensolver()
    exact_calc = GroundStateEigensolver(mapper, numpy_solver)
    exact_result = exact_calc.solve(as_problem)
   
    # solve vqe result
    vqe_result = vqe.compute_minimum_eigenvalue(isa_observable)
    vqe_result_adj = vqe_result.eigenvalue + exact_result.total_energies - exact_result.computed_energies
    
    return vqe_result_adj, exact_result.total_energies

def find_energy(d, backend):

    '''
    Finds ground state energy at various defined nitrogen locations of NH3
    
    Parameters:
    distances (array): location of N in NH3 above or below x-y plane
    rest, optional and see VQE documentation for use
    backend (ibm backend): determines whether to use simulator or not
    Returns:
    vqe energies at each distance (array), exact energy at each distance (array)
    '''
    # create arrays to return
    vqe_energies = []
    exact_energies = []
    
    # define molecule -- nh3 as equilateral planar H3 and N above.
    molecule = 'N 0.0 0.0 {0}; H -0.8121 -0.4689 0.0; H 0.8121 -0.4689 0.0; H 0.0 0.9377 0.0' 
    
    # find the vqe and exact energy at each distance provided
    for i, d in enumerate(distances):
        atom = molecule.format(d)
        isa_circuit, isa_observable, as_problem = get_inputs(atom, backend)
        vqe_result, exact_result = run_VQE(isa_circuit, isa_observable, as_problem, backend)
        vqe_energies.append(vqe_result)
        exact_energies.append(exact_result)
        
        print('vqe energy is', vqe_result, 'at d=', d)
    return vqe_energies, exact_energies

#### 3. Use an available cloud simulator and the above functions to determine ground state energies at various distances.
Warning, expect long run times. Furthermore, IBM cloud computation simulators will soon be depreciated, so this will have to be changed to on-device simulation. Refer to nh3-sdmp.py for references on how to do that.

In [ ]:
'''
This block utilizes the above functions using the simulator
'''
service = QiskitRuntimeService(channel="ibm_quantum") #, token = 'INSERT API TOKEN'
backend = service.get_backend('ibmq_qasm_simulator')
with Session(service=service, backend=backend) as session:
    estimator = Estimator(backend=backend)
    estimator.options.default_shots = 1000
    distances = [0.0]           # Insert what nitrogen distances you would like to find GSE at.
    vqe_results, exact_energies = find_energy(distances, backend)
    session.close()

# Save data in a csv
# Transpose lists to match csv format
rows = zip(distances, vqe_results, exact_energies)
with open('INSERTBACKEND_INSERTDISTANCES_INSERTDATE.csv', 'w') as f:
    writer = csv.writer(f)
    # Write the headers
    writer.writerow(['Distances', 'VQE Energies', 'Exact Energies'])
    # Write the data
    for row in rows:
        writer.writerow(row)

#### 4. Plot simulated noisy results
Note, points [-0.7 -> 0.45] gathered using simulator_mps and [0.5 -> 0.7] using ibm_qasm_simulator due to a wifi loss during calculation.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the csv file
df = pd.read_csv('ibm_simulated_energies_4252024.csv')

# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(df['Distances'], df['VQE Energies'], marker='o')
plt.title('VQE Energies vs Distances')
plt.xlabel('Distances')
plt.ylabel('VQE Energies')
plt.grid(True)
plt.show()

#### 5. Gathering results using IBM quantum computer
To standardize results I choose to use ibm_kyoto.

Expect extremely long runtimes (8+ hours) for a single distance; this is due to being pushed to the back of the line every time the vqe gets a new result from ibm kyoto that is must optimize and not something that can be fixed.

In [ ]:
'''
This block utilizes the above functions using a quantum computer (I chose ibm_kyoto for all points in plots)
'''
service = QiskitRuntimeService(channel="ibm_quantum", token = 'INSERT API TOKEN')
backend = service.get_backend('ibm_kyoto')
with Session(service=service, backend=backend) as session:
    estimator = Estimator(backend=backend)
    estimator.options.default_shots = 1000
    distances = [-0.4, 0.0, 0.4] 
    vqe_results, exact_energies = find_energy(distances, backend)
    session.close()

# Save data in a csv
# Transpose lists to match csv format
rows = zip(distances, vqe_results, exact_energies)
with open('ibm_kyoto_INSERTDISTANCES_INSERTDATE.csv', 'w') as f:
    writer = csv.writer(f)
    # Write the headers
    writer.writerow(['Distances', 'VQE Energies', 'Exact Energies'])
    # Write the data
    for row in rows:
        writer.writerow(row)

### Bug Tracker
- [x] estimator.options.default_shots = 1000 does not actually set the estimator to run 1000 shots
    - Default shots originally is 4000, this is too computationally expensive for the 10 minutes a month IBM gives.
    - Low priority as the code is fully functional, this would just help to approve efficiency in exchange for accuracy
    - Update 4/27, this is not resolved but after seeing low accuracy even with higher number of shots, will not fix this issue.
- [x] Some quantum simulators do not allow the pass manager to run
    - Current fix is to just rerun as the bug occurs before connecting to cloud so does not waste computation time.
    - Next step is to identify which simulators do and do not work then find literature on why.
- [x] All functions work with ibm_qasm_simulator but not with ibm_sherbrooke (Ansatz has no num_qubits setter error).
    - Unsure what changed, but a kernal restart (despite this being a consistent problem) solved it for now. Will reopen if I run into it.
- [x] Qiskit vqe.py import is bugged ~ Estimator accepts 2 position arguments but 4 were given bug
    -   Found out that the EstimatorV2() framework has not been updated to new vqe.py qiskit 1.0.2 compatibility so reverted to EstimatorV1() in imports.
- [x] Many imports are bugged (specifically runtime and optimizer related)
    - Updated all, post-qiskit 1.0 basically every dependency changed--manually found all dependencies in qiskit documentation and added.
- [x] Unexpected results given ammonia geometry
    - Cartesian placement of atoms in ammonia found online was computationally difficult, so calculated coordinates by hand to allow hydrogens to live in x-y plane and nitrogen to be at (0,0, d).
- [x] QuantumInstance class not working
    - QuantumInstance was depreciated, so every tutorial using it is outdated and misleading. Instead, use session/backend joint framework
- [x] Active space transformer not correctly building active space 
    - Solved by changing how parameters of the function were fed to it -- resource I found on it was outdated and had swapped the orbital and electron parameter locations.